# GTHA housing market database
# OSEMN methodology Step 2: Scrub
# Data quality issues discovered in the Teranet dataset

---

This notebook presents examples of data quality issues discovered in the Teranet dataset.

Previous steps included: 

* Step 2.1 

    * the spatial join between the Teranet points and the polygons of GTHA Dissemination Areas (DAs)
    
    * During step 2.1, Teranet records whose coordinates fall outside of the GTHA boundary (as defined by the DA geometry) have been filtered out (6,803,691 of the original 9,039,241 Teranet records remain in the dataset)
     
    * In addition to that, three new columns (`OBJECTID`, `DAUID`, and `CSDNAME`) derived from DA attributes have been added to each Teranet transaction

---

## Import dependencies

In [1]:
import pandas as pd
import os
from time import time

## Multiple transactions

In [2]:
data_path = '../../data/teranet/'
os.listdir(data_path)

['Teranet_with_DA_cols.csv', 'HHSaleHistory.csv']

## Load Teranet data

In [3]:
t = time()
#df = pd.read_csv(data_path + 'Teranet_with_DA_cols.csv',
#                 parse_dates=['registration_date'])
df = pd.read_csv(data_path + 'Teranet_with_DA_cols.csv')
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

/home/stepan/anaconda3/envs/gds/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


----- DataFrame loaded
in 13.60 seconds
with 6,803,691 rows
and 18 columns
-- Column names:
 Index(['lro_num', 'pin', 'consideration_amt', 'registration_date',
       'POSTAL_CODE', 'PROVINCE', 'UNITNO', 'STREET_NAME',
       'STREET_DESIGNATION', 'STREET_DIRECTION', 'MUNICIPALITY',
       'STREET_SUFFIX', 'STREET_NUMBER', 'X', 'Y', 'OBJECTID', 'DAUID',
       'CSDNAME'],
      dtype='object')


In [4]:
df.columns = df.columns.str.lower()
df.columns

Index(['lro_num', 'pin', 'consideration_amt', 'registration_date',
       'postal_code', 'province', 'unitno', 'street_name',
       'street_designation', 'street_direction', 'municipality',
       'street_suffix', 'street_number', 'x', 'y', 'objectid', 'dauid',
       'csdname'],
      dtype='object')

## Multiple transactions
There is a number of ways in which multiple transactions can be recorded in the Teranet dataset.

In [6]:
mask1 = df['pin'] == 248580237
df.loc[mask1, ['pin', 'registration_date', 'consideration_amt', 'unitno', 
               'street_name', 'street_designation', 'municipality']]\
    .sort_values('registration_date')

,pin,registration_date,consideration_amt,unitno,street_name,street_designation,municipality
163593,248580237,12/22/2011 0:00:00,5439000.0,NaN,",3525,3535 & 3545 REBECCA STREET",NaN,OAKVILLE
163478,248580237,5/2/2016 0:00:00,10504865.0,NaN,",3525,3535 & 3545 REBECCA STREET",NaN,OAKVILLE
163485,248580237,6/29/2017 0:00:00,2.0,NaN,",3525,3535 & 3545 REBECCA STREET",NaN,OAKVILLE
